## Extrayendo formas de onda de P, S y ruido de estación RUS en determinado intervalo de tiempo

In [127]:
import MySQLdb
import datetime
import json
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import os

ip_fdsn = "http://10.100.100.232"
port_fdsn = "8091"
Format = "mseed"

client = Client(ip_fdsn+":"+port_fdsn)
db = MySQLdb.connect(host="10.100.100.232",    # your host, usually localhost
                     user="consulta",         # your username
                     passwd="consulta",  # your password
                     db="seiscomp3")        # name of the data base
cur = db.cursor()

cur.execute("Select  POEv.publicID,\
 Pick.waveformID_stationCode,\
Pick.phaseHint_code,Pick.phaseHint_used,Pick.evaluationMode,Pick.time_value, Pick.time_value_ms\
 from Event AS EvMF left join PublicObject AS POEv ON EvMF._oid = POEv._oid \
left join PublicObject as POOri ON EvMF.preferredOriginID=POOri.publicID \
left join Origin ON POOri._oid=Origin._oid left join PublicObject as POMag on EvMF.preferredMagnitudeID=POMag.publicID \
left join Magnitude ON Magnitude._oid = POMag._oid \
left join Arrival on Arrival._parent_oid=Origin._oid \
left join PublicObject as POOri1 on POOri1.publicID = Arrival.pickID \
left join Pick on Pick._oid= POOri1._oid \
where \
(Pick.phaseHint_used = 1 AND Pick.phaseHint_code = 'P' AND Pick.evaluationMode = 'manual' AND Pick.waveformID_stationCode = 'RUS' AND Origin.latitude_value between -4 and 14 AND Origin.longitude_value between -82 AND -67 ) \
and Origin.time_value between '2016-01-01 00:00:00' and '2016-12-31 23:59:59'")



17660L

In [128]:
c = cur.fetchall()
# tomando solo las fases P
# fue necesario hacer esto debido a que la consulta en 2016 en el campo phaseHint_code 
# a todas las fases las llama P. Para tomar la P, cogemos la que tiene tiempo menor.
rows = []
for i in range(1,len(c)):
    if c[i][0] == c[i-1][0]:
        if c[i][5] > c[i-1][5]:
            rows.append(c[i-1])

dt=2.56
dt1 = 2*dt-0.5
dt2 = 0.5
folder = "/home/daniel/Data/wf_512_PTB/"
station = "PTB"
count = 0
p_count = 0
s_count = 0

for l in ["P", "S", "N"]:
    if not os.path.exists(folder+l):
        os.makedirs(folder+l)

for row in rows:
    t = UTCDateTime(row[-2]+datetime.timedelta(milliseconds=float(row[-1])/1000))
    
    try:
        # extrayendo forma de onda
        st = client.get_waveforms("CM", station, "00", "HHZ", t-dt, t+dt)
    except:
        print row
        continue

    st.trim(t-dt, t+dt)
    st.merge(fill_value="interpolate")
    
    for tr in st:
        # guardando trazas de P y de S
        count +=1
        print count
        ch = tr.stats.channel
        name = row[2]+"_"+row[0]+"_"+t.strftime("%Y%m%d-%H%M%S")+"_"+ch+".mseed"
        print name
        tr.write(folder+row[2]+"/"+name, Format)
    
    if row[2] == "P":
        # extrayendo forma de onda de ruido de tamaño dt, 3s antes de la P
        st = client.get_waveforms("CM", station, "00", "HHZ", t-4*dt, t-2*dt)
        st.trim(t-4*dt, t-2*dt)
        st.merge(fill_value="interpolate")
        
        #print "len(st)=",len(st)
        for tr in st:
            p_count +=1
            count +=1
            print count
            ch = tr.stats.channel
            name = "N_"+row[0]+"_"+t.strftime("%Y%m%d-%H%M%S")+"_"+ch+".mseed"
            print name
            tr.write(folder+"N/"+name, Format)
    else:
        for tr in st: s_count += 1
        

print "Total sismogramas: %s\nSismogramas P: %s, Sismogramas S: %s"%(count, p_count, s_count)


1
P_SGC2016aaat_20160101-002338_HHZ.mseed
2
N_SGC2016aaat_20160101-002338_HHZ.mseed
3
P_SGC2016aabh_20160101-003952_HHZ.mseed
4
N_SGC2016aabh_20160101-003952_HHZ.mseed
5
P_SGC2016aacv_20160101-012558_HHZ.mseed
6
N_SGC2016aacv_20160101-012558_HHZ.mseed
7
P_SGC2016aahn_20160101-034822_HHZ.mseed
8
N_SGC2016aahn_20160101-034822_HHZ.mseed
9
P_SGC2016aakl_20160101-051629_HHZ.mseed
10
N_SGC2016aakl_20160101-051629_HHZ.mseed
11
P_SGC2016aakw_20160101-052921_HHZ.mseed
12
N_SGC2016aakw_20160101-052921_HHZ.mseed
13
P_SGC2016aauc_20160101-101003_HHZ.mseed
14
N_SGC2016aauc_20160101-101003_HHZ.mseed
15
P_SGC2016aaws_20160101-112822_HHZ.mseed
16
N_SGC2016aaws_20160101-112822_HHZ.mseed
17
P_SGC2016abcy_20160101-143813_HHZ.mseed
18
N_SGC2016abcy_20160101-143813_HHZ.mseed
19
P_SGC2016abfe_20160101-154550_HHZ.mseed
20
N_SGC2016abfe_20160101-154550_HHZ.mseed
21
P_SGC2016abgc_20160101-161254_HHZ.mseed
22
N_SGC2016abgc_20160101-161254_HHZ.mseed
23
P_SGC2016abnm_20160101-195707_HHZ.mseed
24
N_SGC2016abnm_201

198
N_SGC2016ajlm_20160106-040127_HHZ.mseed
199
P_SGC2016ajra_20160106-064915_HHZ.mseed
200
N_SGC2016ajra_20160106-064915_HHZ.mseed
201
P_SGC2016ajtj_20160106-080121_HHZ.mseed
202
N_SGC2016ajtj_20160106-080121_HHZ.mseed
203
P_SGC2016ajuq_20160106-083905_HHZ.mseed
204
N_SGC2016ajuq_20160106-083905_HHZ.mseed
205
P_SGC2016ajuz_20160106-084933_HHZ.mseed
206
N_SGC2016ajuz_20160106-084933_HHZ.mseed
207
P_SGC2016akaf_20160106-112832_HHZ.mseed
208
N_SGC2016akaf_20160106-112832_HHZ.mseed
209
P_SGC2016akbh_20160106-120100_HHZ.mseed
210
N_SGC2016akbh_20160106-120100_HHZ.mseed
211
P_SGC2016akbz_20160106-122214_HHZ.mseed
212
N_SGC2016akbz_20160106-122214_HHZ.mseed
213
P_SGC2016akcp_20160106-124047_HHZ.mseed
214
N_SGC2016akcp_20160106-124047_HHZ.mseed
215
P_SGC2016akjl_20160106-160837_HHZ.mseed
216
N_SGC2016akjl_20160106-160837_HHZ.mseed
217
P_SGC2016aknc_20160106-175841_HHZ.mseed
218
N_SGC2016aknc_20160106-175841_HHZ.mseed
219
P_SGC2016akro_20160106-201421_HHZ.mseed
220
N_SGC2016akro_20160106-20142

386
N_SGC2016aspu_20160111-042607_HHZ.mseed
387
P_SGC2016astq_20160111-062236_HHZ.mseed
388
N_SGC2016astq_20160111-062236_HHZ.mseed
389
P_SGC2016asum_20160111-064737_HHZ.mseed
390
N_SGC2016asum_20160111-064737_HHZ.mseed
391
P_SGC2016aszl_20160111-091821_HHZ.mseed
392
N_SGC2016aszl_20160111-091821_HHZ.mseed
393
P_SGC2016atgq_20160111-125619_HHZ.mseed
394
N_SGC2016atgq_20160111-125619_HHZ.mseed
395
P_SGC2016atmw_20160111-160531_HHZ.mseed
396
N_SGC2016atmw_20160111-160531_HHZ.mseed
397
P_SGC2016atxb_20160111-211512_HHZ.mseed
398
N_SGC2016atxb_20160111-211512_HHZ.mseed
399
P_SGC2016auaf_20160111-224951_HHZ.mseed
400
N_SGC2016auaf_20160111-224951_HHZ.mseed
401
P_SGC2016audm_20160112-002932_HHZ.mseed
402
N_SGC2016audm_20160112-002932_HHZ.mseed
403
P_SGC2016aueq_20160112-010439_HHZ.mseed
404
N_SGC2016aueq_20160112-010439_HHZ.mseed
405
P_SGC2016auey_20160112-011256_HHZ.mseed
406
N_SGC2016auey_20160112-011256_HHZ.mseed
407
P_SGC2016aufc_20160112-011753_HHZ.mseed
408
N_SGC2016aufc_20160112-01175

578
N_SGC2016bbxz_20160116-064738_HHZ.mseed
579
P_SGC2016bcbj_20160116-083018_HHZ.mseed
580
N_SGC2016bcbj_20160116-083018_HHZ.mseed
581
P_SGC2016bcco_20160116-090654_HHZ.mseed
582
N_SGC2016bcco_20160116-090654_HHZ.mseed
583
P_SGC2016bcki_20160116-130216_HHZ.mseed
584
N_SGC2016bcki_20160116-130216_HHZ.mseed
585
P_SGC2016bcrm_20160116-163906_HHZ.mseed
586
N_SGC2016bcrm_20160116-163906_HHZ.mseed
587
P_SGC2016bctv_20160116-174924_HHZ.mseed
588
N_SGC2016bctv_20160116-174924_HHZ.mseed
589
P_SGC2016bcuj_20160116-180634_HHZ.mseed
590
N_SGC2016bcuj_20160116-180634_HHZ.mseed
591
P_SGC2016bcym_20160116-201038_HHZ.mseed
592
N_SGC2016bcym_20160116-201038_HHZ.mseed
593
P_SGC2016bdbf_20160116-213347_HHZ.mseed
594
N_SGC2016bdbf_20160116-213347_HHZ.mseed
595
P_SGC2016bdbv_20160116-215152_HHZ.mseed
596
N_SGC2016bdbv_20160116-215152_HHZ.mseed
597
P_SGC2016bddg_20160116-223543_HHZ.mseed
598
N_SGC2016bddg_20160116-223543_HHZ.mseed
599
P_SGC2016bdee_20160116-230350_HHZ.mseed
600
N_SGC2016bdee_20160116-23035

765
P_SGC2016bisu_20160120-000727_HHZ.mseed
766
N_SGC2016bisu_20160120-000727_HHZ.mseed
767
P_SGC2016bixm_20160120-022938_HHZ.mseed
768
N_SGC2016bixm_20160120-022938_HHZ.mseed
769
P_SGC2016biyk_20160120-025738_HHZ.mseed
770
N_SGC2016biyk_20160120-025738_HHZ.mseed
771
P_SGC2016bjez_20160120-061755_HHZ.mseed
772
N_SGC2016bjez_20160120-061755_HHZ.mseed
773
P_SGC2016bjgs_20160120-070908_HHZ.mseed
774
N_SGC2016bjgs_20160120-070908_HHZ.mseed
775
P_SGC2016bjig_20160120-075609_HHZ.mseed
776
N_SGC2016bjig_20160120-075609_HHZ.mseed
777
P_SGC2016bjof_20160120-105711_HHZ.mseed
778
N_SGC2016bjof_20160120-105711_HHZ.mseed
779
P_SGC2016bkgl_20160120-200906_HHZ.mseed
780
N_SGC2016bkgl_20160120-200906_HHZ.mseed
781
P_SGC2016bkka_20160120-215809_HHZ.mseed
782
N_SGC2016bkka_20160120-215809_HHZ.mseed
783
P_SGC2016bklt_20160120-225040_HHZ.mseed
784
N_SGC2016bklt_20160120-225040_HHZ.mseed
785
P_SGC2016bkmp_20160120-231629_HHZ.mseed
786
N_SGC2016bkmp_20160120-231629_HHZ.mseed
787
P_SGC2016bknk_20160120-23400

...

8508
N_SGC2016ncnj_20160705-085955_HHZ.mseed
8509
P_SGC2016ncor_20160705-093953_HHZ.mseed
8510
N_SGC2016ncor_20160705-093953_HHZ.mseed
8511
P_SGC2016ncpw_20160705-101605_HHZ.mseed
8512
N_SGC2016ncpw_20160705-101605_HHZ.mseed
8513
P_SGC2016ncqh_20160705-102824_HHZ.mseed
8514
N_SGC2016ncqh_20160705-102824_HHZ.mseed
8515
P_SGC2016ncrj_20160705-110122_HHZ.mseed
8516
N_SGC2016ncrj_20160705-110122_HHZ.mseed
8517
P_SGC2016ncsb_20160705-112140_HHZ.mseed
8518
N_SGC2016ncsb_20160705-112140_HHZ.mseed
8519
P_SGC2016nctk_20160705-120240_HHZ.mseed
8520
N_SGC2016nctk_20160705-120240_HHZ.mseed
8521
P_SGC2016ncud_20160705-122450_HHZ.mseed
8522
N_SGC2016ncud_20160705-122450_HHZ.mseed
8523
P_SGC2016ncur_20160705-124105_HHZ.mseed
8524
N_SGC2016ncur_20160705-124105_HHZ.mseed
8525
P_SGC2016ncvn_20160705-130649_HHZ.mseed
8526
N_SGC2016ncvn_20160705-130649_HHZ.mseed
8527
P_SGC2016ncwa_20160705-132157_HHZ.mseed
8528
N_SGC2016ncwa_20160705-132157_HHZ.mseed
8529
P_SGC2016ndou_20160705-225102_HHZ.mseed
8530
N_SGC

8698
N_SGC2016nlcw_20160710-015455_HHZ.mseed
8699
P_SGC2016nldp_20160710-021732_HHZ.mseed
8700
N_SGC2016nldp_20160710-021732_HHZ.mseed
8701
P_SGC2016nlfu_20160710-032320_HHZ.mseed
8702
N_SGC2016nlfu_20160710-032320_HHZ.mseed
8703
P_SGC2016nlla_20160710-060228_HHZ.mseed
8704
N_SGC2016nlla_20160710-060228_HHZ.mseed
8705
P_SGC2016nlnk_20160710-071446_HHZ.mseed
8706
N_SGC2016nlnk_20160710-071446_HHZ.mseed
8707
P_SGC2016nlow_20160710-075843_HHZ.mseed
8708
N_SGC2016nlow_20160710-075843_HHZ.mseed
8709
P_SGC2016nlqh_20160710-084158_HHZ.mseed
8710
N_SGC2016nlqh_20160710-084158_HHZ.mseed
8711
P_SGC2016nlqw_20160710-085914_HHZ.mseed
8712
N_SGC2016nlqw_20160710-085914_HHZ.mseed
8713
P_SGC2016nlqy_20160710-090202_HHZ.mseed
8714
N_SGC2016nlqy_20160710-090202_HHZ.mseed
8715
P_SGC2016nlwp_20160710-115308_HHZ.mseed
8716
N_SGC2016nlwp_20160710-115308_HHZ.mseed
8717
P_SGC2016nmbh_20160710-141513_HHZ.mseed
8718
N_SGC2016nmbh_20160710-141513_HHZ.mseed
8719
P_SGC2016nmcb_20160710-143824_HHZ.mseed
8720
N_SGC

8888
N_SGC2016nsgk_20160713-233851_HHZ.mseed
('SGC2016nshg', 'RUS', 'P', 1, 'manual', datetime.datetime(2016, 7, 14, 0, 5, 10), 570000L)
8889
P_SGC2016nshj_20160714-000841_HHZ.mseed
8890
N_SGC2016nshj_20160714-000841_HHZ.mseed
8891
P_SGC2016nshw_20160714-002348_HHZ.mseed
8892
N_SGC2016nshw_20160714-002348_HHZ.mseed
8893
P_SGC2016nsjc_20160714-010124_HHZ.mseed
8894
N_SGC2016nsjc_20160714-010124_HHZ.mseed
8895
P_SGC2016nsnj_20160714-031003_HHZ.mseed
8896
N_SGC2016nsnj_20160714-031003_HHZ.mseed
8897
P_SGC2016nsqe_20160714-043558_HHZ.mseed
8898
N_SGC2016nsqe_20160714-043558_HHZ.mseed
8899
P_SGC2016nsse_20160714-053555_HHZ.mseed
8900
N_SGC2016nsse_20160714-053555_HHZ.mseed
8901
P_SGC2016nstp_20160714-061854_HHZ.mseed
8902
N_SGC2016nstp_20160714-061854_HHZ.mseed
8903
P_SGC2016nsua_20160714-063218_HHZ.mseed
8904
N_SGC2016nsua_20160714-063218_HHZ.mseed
8905
P_SGC2016nsvu_20160714-072536_HHZ.mseed
8906
N_SGC2016nsvu_20160714-072536_HHZ.mseed
8907
P_SGC2016nswi_20160714-074143_HHZ.mseed
8908
N_S

9075
P_SGC2016nyeb_20160717-051704_HHZ.mseed
9076
N_SGC2016nyeb_20160717-051704_HHZ.mseed
9077
P_SGC2016nyfa_20160717-054630_HHZ.mseed
9078
N_SGC2016nyfa_20160717-054630_HHZ.mseed
9079
P_SGC2016nyff_20160717-055212_HHZ.mseed
9080
N_SGC2016nyff_20160717-055212_HHZ.mseed
9081
P_SGC2016nyge_20160717-062154_HHZ.mseed
9082
N_SGC2016nyge_20160717-062154_HHZ.mseed
9083
P_SGC2016nyjl_20160717-080017_HHZ.mseed
9084
N_SGC2016nyjl_20160717-080017_HHZ.mseed
9085
P_SGC2016nypa_20160717-104959_HHZ.mseed
9086
N_SGC2016nypa_20160717-104959_HHZ.mseed
9087
P_SGC2016nyqw_20160717-114458_HHZ.mseed
9088
N_SGC2016nyqw_20160717-114458_HHZ.mseed
9089
P_SGC2016nytw_20160717-131643_HHZ.mseed
9090
N_SGC2016nytw_20160717-131643_HHZ.mseed
9091
P_SGC2016nyyw_20160717-154816_HHZ.mseed
9092
N_SGC2016nyyw_20160717-154816_HHZ.mseed
9093
P_SGC2016nzio_20160717-204129_HHZ.mseed
9094
N_SGC2016nzio_20160717-204129_HHZ.mseed
9095
P_SGC2016nzme_20160717-223043_HHZ.mseed
9096
N_SGC2016nzme_20160717-223043_HHZ.mseed
9097
P_SGC

KeyboardInterrupt: 

In [19]:
c= rows[1]
dt =1
t = UTCDateTime(c[-2]+datetime.timedelta(milliseconds=float(c[-1])/1000))

# extrayendo forma de onda
st = client.get_waveforms("CM", "RUS", "00", "HH*", t-dt, t+dt)
st.trim(t-dt, t+dt)
st.merge(fill_value="interpolate")

print t.strftime("%Y%m%d-%H%M%S")
print st

20160101-002357
3 Trace(s) in Stream:
CM.RUS.00.HHZ | 2016-01-01T00:23:56.588393Z - 2016-01-01T00:23:58.588393Z | 100.0 Hz, 201 samples
CM.RUS.00.HHE | 2016-01-01T00:23:56.588393Z - 2016-01-01T00:23:58.588393Z | 100.0 Hz, 201 samples
CM.RUS.00.HHN | 2016-01-01T00:23:56.588393Z - 2016-01-01T00:23:58.588393Z | 100.0 Hz, 201 samples


## Consulta para mapa

In [129]:
cur.execute("Select  POEv.publicID,\
 Pick.waveformID_stationCode,\
Pick.phaseHint_code,Pick.phaseHint_used,Pick.evaluationMode,Pick.time_value, Pick.time_value_ms\
 from Event AS EvMF left join PublicObject AS POEv ON EvMF._oid = POEv._oid \
left join PublicObject as POOri ON EvMF.preferredOriginID=POOri.publicID \
left join Origin ON POOri._oid=Origin._oid left join PublicObject as POMag on EvMF.preferredMagnitudeID=POMag.publicID \
left join Magnitude ON Magnitude._oid = POMag._oid \
left join Arrival on Arrival._parent_oid=Origin._oid \
left join PublicObject as POOri1 on POOri1.publicID = Arrival.pickID \
left join Pick on Pick._oid= POOri1._oid \
where \
(Pick.phaseHint_used = 1 AND Pick.phaseHint_code = 'P' AND Pick.evaluationMode = 'manual' AND Pick.waveformID_stationCode = 'RUS' AND Origin.latitude_value between -4 and 14 AND Origin.longitude_value between -82 AND -67 ) \
and Origin.time_value between '2016-01-01 00:00:00' and '2016-12-31 23:59:59'")

17660L

In [130]:
c = cur.fetchall()

In [131]:
len(c)

17660

In [133]:
c_new = []
for i in range(1,len(c)):
    if c[i][0] == c[i-1][0]:
        if c[i][5] > c[i-1][5]:
            c_new.append(c[i-1])

print(len(c),len(c_new))

(17660, 8526)


In [100]:
c

(('SGC2016aaat',
  'RUS',
  'P',
  1,
  'manual',
  datetime.datetime(2016, 1, 1, 0, 23, 38),
  990000L),
 ('SGC2016aaat',
  'RUS',
  'P',
  1,
  'manual',
  datetime.datetime(2016, 1, 1, 0, 23, 57),
  590000L),
 ('SGC2016aabh',
  'RUS',
  'P',
  1,
  'manual',
  datetime.datetime(2016, 1, 1, 0, 39, 52),
  150000L),
 ('SGC2016aabh',
  'RUS',
  'P',
  1,
  'manual',
  datetime.datetime(2016, 1, 1, 0, 40, 10),
  780000L),
 ('SGC2016aacv',
  'RUS',
  'P',
  1,
  'manual',
  datetime.datetime(2016, 1, 1, 1, 25, 58),
  780000L),
 ('SGC2016aacv',
  'RUS',
  'P',
  1,
  'manual',
  datetime.datetime(2016, 1, 1, 1, 26, 17),
  70000L),
 ('SGC2016aahn',
  'RUS',
  'P',
  1,
  'manual',
  datetime.datetime(2016, 1, 1, 3, 48, 22),
  150000L),
 ('SGC2016aahn',
  'RUS',
  'P',
  1,
  'manual',
  datetime.datetime(2016, 1, 1, 3, 48, 40),
  590000L),
 ('SGC2016aakl',
  'RUS',
  'P',
  1,
  'manual',
  datetime.datetime(2016, 1, 1, 5, 16, 29),
  280000L),
 ('SGC2016aakl',
  'RUS',
  'P',
  1,
  'manual

In [ ]:
[16:18]